In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

In [7]:
data = pd.read_csv("checkpoint_1.csv")

data_size_before = len(data)

data = data.drop_duplicates()
nan_count_series = data.isnull().sum(axis=1)

data["nun_cnt"] = nan_count_series
data = data[data["nun_cnt"] <= len(data.columns) - 2]
data_size_after = len(data)

data = data.fillna(method = 'ffill', axis = 0)
data = data.fillna(0, axis = 0)

data = data.drop(columns="nun_cnt")

C:\Users\sahab\AppData\Local\Temp\ipykernel_2000\4228363583.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method = 'ffill', axis = 0)


In [13]:
data["Дата и время"] = pd.to_datetime(data["Дата и время"])

In [14]:
df_weekly = data.groupby(
    "tractor"
).apply(
    lambda x: x.set_index("Дата и время").resample("W").mean()
).reset_index()

df_monthly = data.groupby(
    'tractor'
).apply(
    lambda x: x.set_index("Дата и время").resample("M").mean()
).reset_index()


C:\Users\sahab\AppData\Local\Temp\ipykernel_2000\3042662231.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  lambda x: x.set_index("Дата и время").resample("M").mean()
C:\Users\sahab\AppData\Local\Temp\ipykernel_2000\3042662231.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  lambda x: x.set_index("Дата и время").resample("M").mean()


In [25]:
df_monthly = df_monthly.interpolate()
df_monthly = df_monthly.sort_values("Дата и время")


C:\Users\sahab\AppData\Local\Temp\ipykernel_2000\1556321869.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_monthly = df_monthly.interpolate()


In [27]:
df_weekly = df_weekly.interpolate()
df_weekly = df_weekly.sort_values("Дата и время")

C:\Users\sahab\AppData\Local\Temp\ipykernel_2000\1881610047.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_weekly = df_weekly.interpolate()


In [51]:
def get_predict_for_time_sample(
        tractor_id: int, data: pd.DataFrame,
        time_sample_type: str
    ) -> dict[str, str]:

    data_for_train = data[
        data["tractor"] == tractor_id
    ].drop(
        columns=[
            "Дата и время",
            "tractor"
        ]
    )
    
    clf = IsolationForest(
        random_state=42
    ).fit(
        data_for_train
    )

    i_forest_predict = clf.predict(data_for_train)

    data_for_train["label"] = i_forest_predict
    display(data_for_train)

    i_forest_predict_last = i_forest_predict[-1]

    answer_string = "машина рекомендуется к проверке" if i_forest_predict_last == -1 else "машина в порядке"

    if time_sample_type == "w":
        answer_string = "По данным за последнюю неделю " + answer_string
    elif time_sample_type == "m":
        answer_string = "По данным за последний месяц " + answer_string

    return {tractor_id: answer_string}

In [52]:
get_predict_for_time_sample(
    "С2177",
    df_weekly,
    "w"
)

,"Давл.масла двиг.,кПа",КПП. Температура масла,КПП. Давление масла в системе смазки,Скорость,ДВС. Давление смазки,ДВС. Температура охлаждающей жидкости,"Давление в пневмостистеме (spn46), кПа",Электросистема. Напряжение,ДВС. Частота вращения коленчатого вала,Нейтраль КПП (spn3843),...,Засоренность фильтра КПП (spn3847),Аварийное давление масла ДВС (spn3846),Засоренность фильтра ДВС (spn3845),Засоренность фильтра рулевого управления (spn3844),Засоренность фильтра навесного оборудования (spn3851),Аварийная температура масла в гидросистеме (spn3849),Аварийное давление в I контуре тормозной системы (spn3848),Аварийное давление в II контуре тормозной системы (spn3855),Зарядка АКБ (spn3854),label
490,267.095658,-273.0,937.663728,9.595282,267.095658,79.028730,817.099576,22.036263,1210.602381,0.182582,...,0.0,0.005877,0.0,1.000000,0.000000,0.0,0.003510,0.0,0.001877,1
491,252.291252,-273.0,920.476075,9.075509,252.291252,78.417729,815.601800,21.660944,1153.066633,0.194411,...,0.0,0.004880,0.0,1.000000,0.000000,0.0,0.003914,0.0,0.001475,1
492,237.486847,-273.0,903.288421,8.555736,237.486847,77.806729,814.104024,21.285624,1095.530886,0.206240,...,0.0,0.003883,0.0,1.000000,0.000000,0.0,0.004318,0.0,0.001074,1
493,222.682441,-273.0,886.100767,8.035963,222.682441,77.195728,812.606248,20.910304,1037.995138,0.218069,...,0.0,0.002886,0.0,1.000000,0.000000,0.0,0.004723,0.0,0.000672,1
494,207.878036,-273.0,868.913114,7.516190,207.878036,76.584727,811.108473,20.534984,980.459390,0.229897,...,0.0,0.001889,0.0,1.000000,0.000000,0.0,0.005127,0.0,0.000270,1
495,258.997468,-273.0,913.955351,9.429827,258.998389,79.915765,816.940852,22.132612,1143.532825,0.147756,...,0.0,0.072497,0.0,1.000000,0.000000,0.0,0.004603,0.0,0.020944,-1
496,230.380135,-273.0,890.616465,8.010563,230.425922,78.142105,815.145116,21.286768,1021.972453,0.249082,...,0.0,0.049048,0.0,1.000000,0.001524,0.0,0.003069,0.0,0.013962,-1
497,201.762802,-273.0,867.277579,6.591299,201.853456,76.368444,813.349379,20.440924,900.412080,0.350408,...,0.0,0.025600,0.0,1.000000,0.003047,0.0,0.001534,0.0,0.006981,-1
498,173.145469,-273.0,843.938693,5.172035,173.280990,74.594784,811.553643,19.595079,778.851707,0.451734,...,0.0,0.002151,0.0,1.000000,0.004571,0.0,0.000000,0.0,0.000000,-1
499,272.839747,-273.0,978.104538,11.043682,272.839747,79.063469,818.506605,21.796267,1198.502190,0.212522,...,0.0,0.004595,0.0,0.999713,0.000000,0.0,0.005169,0.0,0.000287,1


{'С2177': 'По данным за последнюю неделю машина в порядке'}

In [53]:
get_predict_for_time_sample(
    "С2177",
    df_monthly,
    "m"
)

,"Давл.масла двиг.,кПа",КПП. Температура масла,КПП. Давление масла в системе смазки,Скорость,ДВС. Давление смазки,ДВС. Температура охлаждающей жидкости,"Давление в пневмостистеме (spn46), кПа",Электросистема. Напряжение,ДВС. Частота вращения коленчатого вала,Нейтраль КПП (spn3843),...,Засоренность фильтра КПП (spn3847),Аварийное давление масла ДВС (spn3846),Засоренность фильтра ДВС (spn3845),Засоренность фильтра рулевого управления (spn3844),Засоренность фильтра навесного оборудования (spn3851),Аварийная температура масла в гидросистеме (spn3849),Аварийное давление в I контуре тормозной системы (spn3848),Аварийное давление в II контуре тормозной системы (spn3855),Зарядка АКБ (spn3854),label
121,267.095658,-273.0,937.663728,9.595282,267.095658,79.028730,817.099576,22.036263,1210.602381,0.182582,...,0.0,0.005877,0.0,1.000000,0.000000,0.0,0.003510,0.0,0.001877,1
122,235.466402,-273.0,893.221712,8.548950,235.466899,78.382437,814.256117,21.397199,1068.467597,0.185567,...,0.0,0.039995,0.0,1.000000,0.000000,0.0,0.004844,0.0,0.011427,1
123,221.352034,-273.0,908.813776,8.011235,221.422025,76.755590,814.915706,20.659450,981.771160,0.336064,...,0.0,0.003333,0.0,0.999861,0.002361,0.0,0.002500,0.0,0.000139,-1
124,247.317354,-273.0,882.449771,9.620733,247.372364,79.115914,814.401048,21.963451,1129.704895,0.237328,...,0.0,0.020694,0.0,0.999607,0.000000,0.0,0.002358,0.0,0.011395,-1
125,238.195752,-273.0,999.262249,9.202800,238.209513,78.183618,814.509813,21.503607,1103.793310,0.236470,...,0.0,0.010332,0.0,0.999203,0.000046,0.0,0.002528,0.0,0.003902,1
126,230.550215,-273.0,963.994753,8.097376,230.553468,77.789826,812.832092,21.344858,1045.274763,0.245435,...,0.0,0.017867,0.0,0.998872,0.000000,0.0,0.008225,0.0,0.006152,-1
127,226.327306,-273.0,889.133213,7.926175,226.327306,77.363472,810.837854,21.986686,1022.605787,0.219560,...,0.0,0.015069,0.0,1.000000,0.000000,0.0,0.031193,0.0,0.006932,-1
128,263.794883,-273.0,987.728688,10.231965,263.798367,79.565269,815.111595,22.811415,1250.052205,0.164834,...,0.0,0.006859,0.0,1.000000,0.000000,0.0,0.007512,0.0,0.005335,1
129,226.322075,-273.0,1007.103279,7.684716,226.322075,77.963289,813.975526,21.734569,1023.060955,0.276309,...,0.0,0.008443,0.0,1.000000,0.000000,0.0,0.003916,0.0,0.006241,1
130,260.086103,-273.0,1000.374749,9.696766,260.232880,78.749164,815.265670,22.389053,1210.315442,0.157595,...,0.0,0.002231,0.0,0.999888,0.000000,0.0,0.006915,0.0,0.001227,1


{'С2177': 'По данным за последний месяц машина рекомендуется к проверке'}